In [1]:
import json, requests, os, math
import numpy as np
import pandas as pd
import geopandas as gpd
from dotenv import load_dotenv
from pymongo import MongoClient, GEO2D
from geopy.distance import distance

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

## Filtamos por documentos que tengan los campos funding_round y offices no vacio, que no hayan quebrado

In [3]:
companies = list(db["companies"].find({"funding_rounds":{"$exists": True, "$not": {"$size": 0}},"offices":{"$exists": True, "$not": {"$size": 0}}, "offices.latitude":{"$exists":True, "$ne":None}, "offices.longitude":{"$exists":True, "$ne":None},"deadpooled_year":{"$not":{"$gte":0}}},{"name":1,"offices":1,"category_code":1, "funding_rounds":1, "founded_year":1}))
df = pd.DataFrame(companies)
df.head()

,_id,name,category_code,founded_year,funding_rounds,offices
0,52cdef7c4bab8bd675297d91,Geni,web,2006.0,"[{'id': 6, 'round_code': 'a', 'source_url': ''...","[{'description': 'Headquarters', 'address1': '..."
1,52cdef7c4bab8bd675297d8d,Digg,news,2004.0,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...","[{'description': None, 'address1': '135 Missis..."
2,52cdef7c4bab8bd675297d96,Gizmoz,web,2003.0,"[{'id': 9, 'round_code': 'a', 'source_url': 'h...","[{'description': None, 'address1': None, 'addr..."
3,52cdef7c4bab8bd675297d97,Scribd,news,2007.0,"[{'id': 10, 'round_code': 'seed', 'source_url'...","[{'description': 'HQ', 'address1': '539 Bryant..."
4,52cdef7c4bab8bd675297d99,Lala,games_video,NaN,"[{'id': 16, 'round_code': 'b', 'source_url': '...","[{'description': 'Lala Headquarters', 'address..."


In [4]:
def sum_funding_rounds(funding_rounds):
    currency_dict = {"USD":1, "EUR":1.09, "GBP":1.29, "CAD":0.75, "SEK":0.1, "JPY":0.0091}
    result = []
    for e in df.funding_rounds:
        result.append(sum([0 if k["raised_amount"] == None else k["raised_amount"]*currency_dict[k["raised_currency_code"]] for k in e if "raised_amount" in k.keys() and "raised_currency_code" in k.keys()]))
    return result

In [5]:
total_funding = sum_funding_rounds(df.funding_rounds)

In [6]:
df["Total_funding"] = total_funding

In [7]:
df.head()

,_id,name,category_code,founded_year,funding_rounds,offices,Total_funding
0,52cdef7c4bab8bd675297d91,Geni,web,2006.0,"[{'id': 6, 'round_code': 'a', 'source_url': ''...","[{'description': 'Headquarters', 'address1': '...",16500000.0
1,52cdef7c4bab8bd675297d8d,Digg,news,2004.0,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...","[{'description': None, 'address1': '135 Missis...",45000000.0
2,52cdef7c4bab8bd675297d96,Gizmoz,web,2003.0,"[{'id': 9, 'round_code': 'a', 'source_url': 'h...","[{'description': None, 'address1': None, 'addr...",18100000.0
3,52cdef7c4bab8bd675297d97,Scribd,news,2007.0,"[{'id': 10, 'round_code': 'seed', 'source_url'...","[{'description': 'HQ', 'address1': '539 Bryant...",25762000.0
4,52cdef7c4bab8bd675297d99,Lala,games_video,NaN,"[{'id': 16, 'round_code': 'b', 'source_url': '...","[{'description': 'Lala Headquarters', 'address...",44150000.0


In [8]:
df = df.explode('offices')
df_offices= df[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
df = pd.concat([df,df_offices], axis=1)
df.head()

,_id,name,category_code,founded_year,funding_rounds,offices,Total_funding,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,52cdef7c4bab8bd675297d91,Geni,web,2006.0,"[{'id': 6, 'round_code': 'a', 'source_url': ''...","{'description': 'Headquarters', 'address1': '9...",16500000.0,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064
1,52cdef7c4bab8bd675297d8d,Digg,news,2004.0,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...","{'description': None, 'address1': '135 Mississ...",45000000.0,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523
2,52cdef7c4bab8bd675297d96,Gizmoz,web,2003.0,"[{'id': 9, 'round_code': 'a', 'source_url': 'h...","{'description': None, 'address1': None, 'addre...",18100000.0,None,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472
3,52cdef7c4bab8bd675297d97,Scribd,news,2007.0,"[{'id': 10, 'round_code': 'seed', 'source_url'...","{'description': 'HQ', 'address1': '539 Bryant ...",25762000.0,HQ,539 Bryant Street,,94107,San Francisco,CA,USA,37.789634,-122.404052
4,52cdef7c4bab8bd675297d99,Lala,games_video,NaN,"[{'id': 16, 'round_code': 'b', 'source_url': '...","{'description': 'Lala Headquarters', 'address1...",44150000.0,Lala Headquarters,209 Hamilton Ave,Suite #200,94301,Palo Alto,CA,USA,37.451151,-122.154369


In [9]:
df = df.drop(columns = ["offices","funding_rounds"])

In [10]:
def asGeoJSON(lat,lng):
    try:
        lat = float(lat)
        lng = float(lng)
        if not math.isnan(lat) and not math.isnan(lng):
            return {
                "type":"Point",
                "coordinates":[lng,lat]
            }
    except Exception:
        print("Invalid data")
        return None

In [11]:
df["location"] = df[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
df = df.drop(columns=["_id","address1", "address2","zip_code"])

In [12]:
df.head()

,name,category_code,founded_year,Total_funding,description,city,state_code,country_code,latitude,longitude,location
0,Geni,web,2006.0,16500000.0,Headquarters,West Hollywood,CA,USA,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
1,Digg,news,2004.0,45000000.0,None,San Francisco,CA,USA,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
2,Gizmoz,web,2003.0,18100000.0,None,Menlo Park,CA,USA,37.484130,-122.169472,"{'type': 'Point', 'coordinates': [-122.169472,..."
3,Scribd,news,2007.0,25762000.0,HQ,San Francisco,CA,USA,37.789634,-122.404052,"{'type': 'Point', 'coordinates': [-122.404052,..."
4,Lala,games_video,NaN,44150000.0,Lala Headquarters,Palo Alto,CA,USA,37.451151,-122.154369,"{'type': 'Point', 'coordinates': [-122.154369,..."


In [13]:
df.to_json("OUTPUT/cleaned_offices.json", orient="records")

In [14]:
def create_document(database, json_document, new_collection, geoindex=False):
    client = MongoClient("mongodb://localhost/")
    db = client[database]
    db[new_collection].drop()
    companies_clean = db[new_collection]
    with open(json_document) as f:
        file_data = json.load(f)
    companies_clean.insert_many(file_data)
    if geoindex:
        companies_clean.create_index([("location", "2dsphere")])
    client.close()

In [15]:
create_document("companies", 'OUTPUT/cleaned_offices.json', 'companies_clean', True)

In [16]:
def geocode(address):
    data = requests.get(f"https://geocode.xyz/{address}?json=1").json()
    return {
        "type":"Point",
        "coordinates":[float(data["longt"]),float(data["latt"])]
    }
geocode("new york")

{'type': 'Point', 'coordinates': [-73.9586, 40.68908]}

In [17]:
# https://docs.mongodb.com/manual/reference/operator/query/near/
def withGeoQuery(location,maxDistance=10000,minDistance=0,field="location"):
    return {
       field: {
         "$near": {
           "$geometry": location if type(location)==dict else geocode(location),
           "$maxDistance": maxDistance,
           "$minDistance": minDistance
         }
       }
    }
query = withGeoQuery("new york")
query

{'location': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [-73.9586, 40.68908]},
   '$maxDistance': 10000,
   '$minDistance': 0}}}

In [18]:
comp = list(db["companies_clean"].find(query))

In [19]:
df = pd.DataFrame(comp)
df

,_id,name,category_code,founded_year,Total_funding,description,city,state_code,country_code,latitude,longitude,location
0,5e4088732f0da6a948f0497c,Etsy,ecommerce,2005.0,91650000.0,,Brooklyn,NY,USA,40.694731,-73.983268,"{'type': 'Point', 'coordinates': [-73.983268, ..."
1,5e4088732f0da6a948f04dde,Kidos,games_video,2011.0,200000.0,Corporate Headquarters,Brooklyn,NY,USA,40.714056,-73.957854,"{'type': 'Point', 'coordinates': [-73.957854, ..."
2,5e4088732f0da6a948f04d5b,Kidos,games_video,2011.0,200000.0,Corporate Headquarters,Brooklyn,NY,USA,40.714056,-73.957854,"{'type': 'Point', 'coordinates': [-73.957854, ..."
3,5e4088732f0da6a948f04bda,TuneCore,games_video,2005.0,7000000.0,,Brooklyn,NY,USA,40.704382,-73.986572,"{'type': 'Point', 'coordinates': [-73.986572, ..."
4,5e4088732f0da6a948f04d1a,Pond5,ecommerce,2006.0,500000.0,Pond5,Brooklyn,NY,USA,40.703288,-73.987853,"{'type': 'Point', 'coordinates': [-73.9878525,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
218,5e4088732f0da6a948f0488f,OrganizedWisdom,other,2007.0,2800000.0,None,New York,NY,USA,40.775537,-73.981709,"{'type': 'Point', 'coordinates': [-73.981709, ..."
219,5e4088732f0da6a948f04d30,Producteev,software,2008.0,1310000.0,Headquarters,New York,NY,USA,40.775309,-73.983656,"{'type': 'Point', 'coordinates': [-73.983656, ..."
220,5e4088732f0da6a948f04db4,Producteev,software,2008.0,1310000.0,Headquarters,New York,NY,USA,40.775309,-73.983656,"{'type': 'Point', 'coordinates': [-73.983656, ..."
221,5e4088732f0da6a948f05279,Stemline Therapeutics,biotech,2003.0,15090000.0,,New York,NY,USA,40.777565,-73.945933,"{'type': 'Point', 'coordinates': [-73.9459335,..."


In [20]:
def get_json_locations(latitude=40.42953, longitude=-3.67932, word_key="", radius=25000, limit=1):
    load_dotenv()
    URL = 'https://api.foursquare.com/v2/venues/explore'
    CLIENT_ID = os.getenv("CLIENT_ID")
    CLIENT_SECRET = os.getenv("CLIENT_SECRET")
    VERSION = "20180323"
    
    params = dict(
      client_id = CLIENT_ID,
      client_secret = CLIENT_SECRET,
      v = VERSION,
      ll=f"{latitude},{longitude}",
      query=word_key,
      radius= radius,
      limit=limit
    )
    
    resp = requests.get(url=(URL), params=params)
    return json.loads(resp.text)

## Preparamos una bbdd de aeropuertos con nuestro csv:

In [21]:
airports = pd.read_csv("INPUT/airport-codes_csv.csv")
airports = airports.drop(columns=["ident", "gps_code", "local_code"])
(airports.coordinates[airports.municipality == "Madrid"]).loc[5050].split(",")[1].strip()

'40.896400451660156'

In [22]:
airports["location"] = airports["coordinates"].apply(lambda x:asGeoJSON(float(x.split(",")[1].strip()),float(x.split(",")[0].strip())))
airports["longitude"] = airports.coordinates.apply(lambda x: float(x.split(",")[0].strip()))
airports["latitude"] = airports.coordinates.apply(lambda x: float(x.split(",")[1].strip()))
airports = airports.drop(columns="coordinates")

In [23]:
airports.head()

,type,name,elevation_ft,continent,iso_country,iso_region,municipality,iata_code,location,longitude,latitude
0,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,NaN,"{'type': 'Point', 'coordinates': [-74.93360137...",-74.933601,40.070801
1,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,NaN,"{'type': 'Point', 'coordinates': [-101.473911,...",-101.473911,38.704022
2,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,NaN,"{'type': 'Point', 'coordinates': [-151.6959991...",-151.695999,59.949200
3,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,NaN,"{'type': 'Point', 'coordinates': [-86.77030181...",-86.770302,34.864799
4,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,"{'type': 'Point', 'coordinates': [-91.254898, ...",-91.254898,35.608700


In [24]:
airports.to_json("OUTPUT/cleaned_airports.json", orient="records")

In [25]:
create_document("airports", 'OUTPUT/cleaned_airports.json', 'cleaned_airports', True)

## Vamos a quedarnos solo con los aeropuertos internacionales, nos es más fácil conseguir los valores de iata validos a través de una consulta pymongo.
## Sobreescribimos la base de datos para tenerla con los datos que queremos

In [26]:
client = MongoClient("mongodb://localhost/airports")
db = client.get_database()
ar = list(db["cleaned_airports"].find({"iata_code":{"$exists": True, "$ne": None}}))
df_ar = pd.DataFrame(ar)
df_ar = df_ar.drop(columns="_id")
df_ar.to_json("OUTPUT/cleaned_airports.json", orient="records")
create_document("airports", 'OUTPUT/cleaned_airports.json', 'cleaned_airports', True)

In [27]:
gdf_airports = gpd.GeoDataFrame(df_ar, geometry=gpd.points_from_xy(df_ar.longitude, df_ar.latitude))
gdf_companies = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

In [28]:
gdf_airports.head()

,type,name,elevation_ft,continent,iso_country,iso_region,municipality,iata_code,location,longitude,latitude,geometry
0,small_airport,Utirik Airport,4.0,OC,MH,MH-UTI,Utirik Island,UTK,"{'type': 'Point', 'coordinates': [169.852005, ...",169.852005,11.222000,POINT (169.85200 11.22200)
1,small_airport,Ocean Reef Club Airport,8.0,None,US,US-FL,Key Largo,OCA,"{'type': 'Point', 'coordinates': [-80.27480316...",-80.274803,25.325399,POINT (-80.27480 25.32540)
2,small_airport,Pilot Station Airport,305.0,None,US,US-AK,Pilot Station,PQS,"{'type': 'Point', 'coordinates': [-162.899994,...",-162.899994,61.934601,POINT (-162.89999 61.93460)
3,small_airport,Crested Butte Airpark,8980.0,None,US,US-CO,Crested Butte,CSE,"{'type': 'Point', 'coordinates': [-106.928341,...",-106.928341,38.851918,POINT (-106.92834 38.85192)
4,small_airport,LBJ Ranch Airport,1515.0,None,US,US-TX,Johnson City,JCY,"{'type': 'Point', 'coordinates': [-98.62249755...",-98.622498,30.251801,POINT (-98.62250 30.25180)


In [30]:
def find_services(services, lat, long, radius=30000, limit=1):
    serv = [get_json_locations(lat, long, e, radius, limit) for e in services]
    serv_dfs = [pd.DataFrame(e["response"]["groups"][0]["items"]) for e in serv]
    serv_final = []
    for i,e in enumerate(serv_dfs):
        if "venue" in e.keys():
            if type(serv_final) == list:
                serv_final = pd.DataFrame(columns = e.venue[i].keys())
            for v in e.venue:
                serv_final = serv_final.append(v, ignore_index = True)
    return serv_final

In [31]:
def get_coordinates(city="", airport=False, companies=[], services=[], radius=30000, limit=10):
    coord = geocode(city)["coordinates"]
    #Buscamos los servicios
    serv_dfs = find_services(services, coord[1], coord[0], radius, limit)
    #Buscamos companies
    #query = withGeoQuery(ny, 30000)
    #Buscamos aeropuertos si procede
    if airport:
        ar = gdf_airports[gdf_airports.municipality == city]
    return serv_dfs

In [32]:
#nursery school
sol = get_coordinates("madrid", services=["nursery school"])

In [33]:
df[df.category_code == "design"]

,_id,name,category_code,founded_year,Total_funding,description,city,state_code,country_code,latitude,longitude,location,geometry


In [149]:
query = withGeoQuery("Berlin", 30000)
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()
comp = list(db["companies_clean"].find(query))
df = pd.DataFrame(comp)

In [35]:
design = df[df.category_code=="design"]

In [36]:
sol = get_coordinates("Berlin", services=["nursery school"], limit=50, radius=30000)

In [37]:
sol.head()

,id,name,contact,location,categories,verified,stats,beenHere,photos,hereNow,venuePage
0,4cdfe764ffcf370470a81c82,INA.KINDER.GARTEN Dresdener Straße,{},"{'address': 'Dresdener Straße 128', 'lat': 52....","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
1,52385ad011d2790ab72b2582,Kindergärten City,{},"{'lat': 52.50045546510848, 'lng': 13.443106804...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
2,51ff4f8a498e883b01f44c10,Kindergarten Kleine Auguststraße,{},"{'address': 'Kleine Auguststr. 10-12', 'lat': ...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
3,4ecbd725f790bc9a81a32e72,BCS Kindergarten in the Park,{},"{'address': 'Invalidenstr. 130', 'lat': 52.531...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
4,5a11ab4a6a59503862e588f3,Wunderhaus,{},"{'lat': 52.53271959459243, 'lng': 13.416940178...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN


In [40]:
arp_berlin = gdf_airports[gdf_airports.municipality == "Berlin"]
arp_berlin = arp_berlin[arp_berlin["type"] == "large_airport"]
arp_berlin

,type,name,elevation_ft,continent,iso_country,iso_region,municipality,iata_code,location,longitude,latitude,geometry
1245,large_airport,Berlin-SchÃ¶nefeld Airport,157.0,EU,DE,DE-BR,Berlin,SXF,"{'type': 'Point', 'coordinates': [13.5225, 52....",13.5225,52.380001,POINT (13.52250 52.38000)
1259,large_airport,Berlin-Tegel Airport,122.0,EU,DE,DE-BE,Berlin,TXL,"{'type': 'Point', 'coordinates': [13.2877, 52....",13.2877,52.559700,POINT (13.28770 52.55970)


In [118]:
def get_distance(a):
    return lambda b: (distance((list(a.latitude)[0], list(a.longitude)[0]), (b.latitude, b.longitude))).km

In [119]:
arp_berlin["distance"] = arp_berlin[["latitude","longitude"]].apply(get_distance(design), axis=1)

In [144]:
arp_berlin = arp_berlin.sort_values(["distance"])
arp_berlin

,type,name,elevation_ft,continent,iso_country,iso_region,municipality,iata_code,location,longitude,latitude,geometry,distance
1259,large_airport,Berlin-Tegel Airport,122.0,EU,DE,DE-BE,Berlin,TXL,"{'type': 'Point', 'coordinates': [13.2877, 52....",13.2877,52.559700,POINT (13.28770 52.55970),12.764448
1245,large_airport,Berlin-SchÃ¶nefeld Airport,157.0,EU,DE,DE-BR,Berlin,SXF,"{'type': 'Point', 'coordinates': [13.5225, 52....",13.5225,52.380001,POINT (13.52250 52.38000),14.165190


In [139]:
sol = get_coordinates("Berlin", services=["nursery school"],limit=10)
sol.head()

,id,name,contact,location,categories,verified,stats,beenHere,photos,hereNow,venuePage
0,4cdfe764ffcf370470a81c82,INA.KINDER.GARTEN Dresdener Straße,{},"{'address': 'Dresdener Straße 128', 'lat': 52....","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
1,52385ad011d2790ab72b2582,Kindergärten City,{},"{'lat': 52.50045546510848, 'lng': 13.443106804...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
2,51ff4f8a498e883b01f44c10,Kindergarten Kleine Auguststraße,{},"{'address': 'Kleine Auguststr. 10-12', 'lat': ...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
3,4ecbd725f790bc9a81a32e72,BCS Kindergarten in the Park,{},"{'address': 'Invalidenstr. 130', 'lat': 52.531...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN
4,5a11ab4a6a59503862e588f3,Wunderhaus,{},"{'lat': 52.53271959459243, 'lng': 13.416940178...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN


In [136]:
def get_distance2(a):
    return lambda b: (distance((list(a.latitude)[0], list(a.longitude)[0]), (b["location"]["lat"], b["location"]["lng"]))).km

In [145]:
sol["distance"]=sol[["location"]].apply(get_distance2(design), axis=1)
sol = sol.sort_values(["distance"])

In [146]:
sol

,id,name,contact,location,categories,verified,stats,beenHere,photos,hereNow,venuePage,distance
1,52385ad011d2790ab72b2582,Kindergärten City,{},"{'lat': 52.50045546510848, 'lng': 13.443106804...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN,0.328917
6,50a64e7ae4b0453ba8c1e4d7,oXxymoron,{},"{'address': 'Scharnweberstrasse 23', 'lat': 52...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN,2.009347
0,4cdfe764ffcf370470a81c82,INA.KINDER.GARTEN Dresdener Straße,{},"{'address': 'Dresdener Straße 128', 'lat': 52....","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN,2.033815
5,4e54c85362e1d1844321ff1a,Bambini Oase,{},"{'address': 'Hans-Otto-Str.', 'lat': 52.532571...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN,3.814250
4,5a11ab4a6a59503862e588f3,Wunderhaus,{},"{'lat': 52.53271959459243, 'lng': 13.416940178...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN,4.305319
2,51ff4f8a498e883b01f44c10,Kindergarten Kleine Auguststraße,{},"{'address': 'Kleine Auguststr. 10-12', 'lat': ...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN,4.632090
3,4ecbd725f790bc9a81a32e72,BCS Kindergarten in the Park,{},"{'address': 'Invalidenstr. 130', 'lat': 52.531...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN,5.384159
7,4e646a2c8877954de90aebd3,Europa-Kindergarten Max und Moritz,{},"{'address': 'Kärntener Str. 27', 'lat': 52.479...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",True,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",{'id': '377814011'},7.061729
8,57d6782b498e432cad3cd335,Bayer Kita,{},"{'lat': 52.541739, 'lng': 13.368018, 'labeledL...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN,7.189907
9,54d35f4d498ebad1a55e89ce,Haus der Familie,{},"{'address': 'Schillerstr. 26', 'crossStreet': ...","[{'id': '4f4533814b9074f6e4fb0107', 'name': 'N...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'groups': []}","{'count': 0, 'summary': 'Nobody here', 'groups...",NaN,9.301469


In [148]:
design

,_id,name,category_code,founded_year,Total_funding,description,city,state_code,country_code,latitude,longitude,location
14,5e4088732f0da6a948f04a33,99designs,design,2008.0,35000000.0,Europe,Berlin,None,DEU,52.49862,13.446903,"{'type': 'Point', 'coordinates': [13.4469031, ..."


In [150]:
df["distance"] = 

,_id,name,category_code,founded_year,Total_funding,description,city,state_code,country_code,latitude,longitude,location
0,5e4088732f0da6a948f052f6,Moviepilot,web,2007.0,7000000.0,German Headquarters,Berlin,None,DEU,52.494727,13.398504,"{'type': 'Point', 'coordinates': [13.398504, 5..."
1,5e4088732f0da6a948f04c03,mokono,advertising,2006.0,0.0,Main office,Berlin,None,DEU,52.500888,13.420817,"{'type': 'Point', 'coordinates': [13.420817, 5..."
2,5e4088732f0da6a948f04afd,alaTest,web,2005.0,0.0,alaTest Germany,Berlin,None,DEU,52.509299,13.390060,"{'type': 'Point', 'coordinates': [13.3900596, ..."
3,5e4088732f0da6a948f0521d,My-Hammer,web,2005.0,3200000.0,MY-HAMMER AG,Berlin,None,DEU,52.509231,13.388361,"{'type': 'Point', 'coordinates': [13.3883606, ..."
4,5e4088732f0da6a948f04fd5,Kaeuferportal,enterprise,2008.0,0.0,Headquarters,Berlin,None,DEU,52.512337,13.393667,"{'type': 'Point', 'coordinates': [13.3936665, ..."
5,5e4088732f0da6a948f04acb,Metaversum,web,2006.0,6260000.0,None,Berlin,None,DEU,52.512249,13.419841,"{'type': 'Point', 'coordinates': [13.419841, 5..."
6,5e4088732f0da6a948f04aa7,babbel,education,2007.0,10000000.0,Lesson Nine GmbH,Berlin,None,DEU,52.489700,13.388610,"{'type': 'Point', 'coordinates': [13.38861, 52..."
7,5e4088732f0da6a948f04e81,Ecato,ecommerce,2005.0,0.0,Berlin Office,Berlin,None,DEU,52.513789,13.387649,"{'type': 'Point', 'coordinates': [13.387649, 5..."
8,5e4088732f0da6a948f04b59,imedo,health,2007.0,0.0,,Berlin,None,DEU,52.519212,13.403108,"{'type': 'Point', 'coordinates': [13.403108, 5..."
9,5e4088732f0da6a948f0501f,Escapio,other,2004.0,0.0,Escapio Berlin,Berlin,None,DEU,52.494307,13.434474,"{'type': 'Point', 'coordinates': [13.434474, 5..."
